# Many simulations at different carrying capacities and switching speeds

With dominant paths displayed alongside their weights.

In [1]:
import pandas as pd
import numpy as np
import math
import os
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [3]:
def hist_carrying_cap(landscape, k_values=[9, 8, 7, 6], num=100, **kargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    data = []
    for k in k_values:
        param = dict(kargs)
        param['carrying_cap'] = 10**k
        param['prob_mutation'] = 10**(-(k-1))
        times = [simulate(landscape, **param)['T_f'] for i in range(num)]
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            #xbins = dict(
            #    start = 0,
            #    end = 1000,
            #    size = 10),
            #autobinx = False,
            opacity=0.75,
            name = 'K=10^{}'.format(k)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different carrying capacities'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## Using histograms to display effect of carrying capacity

In [4]:
hist_carrying_cap(dataset2.loc['CPD'], [9, 7, 5, 3])
hist_carrying_cap(pairs[0], [9, 7, 5, 3], frequency=100)

In [4]:
def hist_switching(landscapes, frequencies=[200, 100, 50], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            xbins = dict(
                size = 1),
            autobinx = False,
            opacity=0.75,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            #rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## High switching speeds show large variation in T<sub>f</sub>

Histograms may be less useful in this case. See the box plots below.

In [6]:
hist_switching(pairs[0])
hist_switching(pairs[1])
hist_switching(pairs[2])

In [5]:
def box_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Box(
            y = times,
            boxpoints = False,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Switching speed',
        ),
        yaxis = dict(
            title = 'Time to fixation'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [6]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    T_f_sum = 0
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
            T_f_sum += results['T_f']
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if not np.isnan(max_weight):
        dominant_path = df.iloc[df['Weight'].idxmax()]['Pathway']
    else:
        dominant_path = ""
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            'success_rate': num_successful_paths / num_paths,
            'dominant_path': dominant_path
        }

In [7]:
def table_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    success_counts = []
    success_rates = []
    weights = []
    doms = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, **param)
        success_counts.append(analysis['success_count'])
        success_rates.append(analysis['success_rate'])
        weights.append(analysis['max_weight'])
        doms.append(analysis['dominant_path'])
    df = pd.DataFrame({
        'f': frequencies,
        'Success count': success_counts,
        'Success rate': success_rates,
        'Weight of dominant path': weights,
        'Dominant path': doms
    }).set_index('f')
    return df

## CTX + SAM

### K=10^9

In [10]:
display(table_switching(pairs[0]))
box_switching(pairs[0])

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
175,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
150,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
125,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
100,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
75,100,1.000000,1.000000,"0000,0010,0011,1011,1111"
50,100,1.000000,0.900000,"0000,0010,0011,1011,1111"
25,85,0.500000,1.000000,"0000,0010,1010,1011,1111"
20,11,0.666667,0.909091,"0000,0010,1010,1011,1111"


### K=10^6

In [14]:
display(table_switching(pairs[0], carrying_cap=10**7))
box_switching(pairs[0], carrying_cap=10**7)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,94,0.428571,0.510638,"0000,0010,0011,1011,1111"
175,95,0.500000,0.494737,"0000,0100,0110,1110,1111"
150,95,0.571429,0.421053,"0000,0100,0110,1110,1111"
125,90,0.571429,0.422222,"0000,0100,0110,1110,1111"
100,96,0.750000,0.375000,"0000,0010,0011,1011,1111"
75,93,0.700000,0.408602,"0000,0100,0110,1110,1111"
50,39,0.625000,0.717949,"0000,0010,1010,1011,1111"
25,9,0.428571,0.666667,"0000,0001,0011,1011,1111"
20,6,0.400000,0.833333,"0000,0001,0011,1011,1111"


### K=10^4

In [12]:
display(table_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.000000,0.540000,"0000,0010,0011,0111,1111"
175,100,1.000000,0.510000,"0000,0010,0011,0111,1111"
150,100,1.000000,0.470000,"0000,0010,0011,0111,1111"
125,100,1.000000,0.470000,"0000,0010,0011,0111,1111"
100,100,1.000000,0.490000,"0000,0010,0011,0111,1111"
75,100,1.000000,0.530000,"0000,0010,0011,0111,1111"
50,100,1.000000,0.450000,"0000,0010,0011,0111,1111"
25,100,1.000000,0.480000,"0000,0010,0011,1011,1111"
20,100,1.000000,0.560000,"0000,0010,0011,1011,1111"


## AM + AMC

### K=10^9

In [9]:
display(table_switching(pairs[1]))
box_switching(pairs[1])

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.000000,0.900000,"0000,0010,1010,1011,1111,1101"
175,100,1.000000,0.910000,"0000,0010,1010,1011,1111,1101"
150,100,1.000000,0.920000,"0000,0010,1010,1011,1111,1101"
125,100,1.000000,0.970000,"0000,0010,1010,1011,1111,1101"
100,100,1.000000,0.810000,"0000,0010,1010,1011,1111,1101"
75,100,1.000000,0.980000,"0000,0001,1001,1101"
50,98,0.666667,0.969388,"0000,0001,1001,1101"
25,9,0.500000,0.777778,"0000,0001,1001,1101"
20,7,0.666667,0.857143,"0000,0001,1001,1101"


### K=10^6

In [12]:
display(table_switching(pairs[1], carrying_cap=10**8))
box_switching(pairs[1], carrying_cap=10**8)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.0,0.950000,"0000,0010,1010,1011,1111,1101"
175,100,1.0,0.900000,"0000,0010,1010,1011,1111,1101"
150,100,1.0,0.940000,"0000,0010,1010,1011,1111,1101"
125,100,1.0,0.920000,"0000,0010,1010,1011,1111,1101"
100,100,1.0,0.940000,"0000,0010,1010,1011,1111,1101"
75,100,1.0,0.970000,"0000,0001,1001,1101"
50,28,0.4,0.785714,"0000,0001,1001,1101"
25,0,0.0,NaN,
20,0,0.0,NaN,


### K=10^4

In [15]:
display(table_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.0,0.23,"0000,0010,1010,1011,1111,1101"
175,100,1.0,0.24,"0000,0001,0101,1101"
150,100,1.0,0.29,"0000,0010,1010,1011,1111,1101"
125,100,1.0,0.24,"0000,0010,1010,1011,1111,1101"
100,100,1.0,0.24,"0000,0010,1010,1011,1111,1101"
75,100,1.0,0.23,"0000,0010,1010,1011,1111,1101"
50,100,1.0,0.27,"0000,0001,0101,1101"
25,100,1.0,0.36,"0000,0001,1001,1101"
20,100,1.0,0.59,"0000,0001,1001,1101"


## ZOX + CXM

### K=10^9

In [16]:
display(table_switching(pairs[2]))
box_switching(pairs[2])

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.0,1.00,"0000,0010,0011,0111"
175,100,1.0,1.00,"0000,0010,0011,0111"
150,100,1.0,1.00,"0000,0010,0011,0111"
125,100,1.0,1.00,"0000,0010,0011,0111"
100,100,1.0,1.00,"0000,0010,0011,0111"
75,100,1.0,1.00,"0000,0010,0011,0111"
50,100,1.0,1.00,"0000,0010,0011,0111"
25,100,1.0,0.99,"0000,0010,0110,0111"
20,100,1.0,0.99,"0000,0010,0110,0111"


### K=10^6

In [31]:
display(table_switching(pairs[2], frequencies=[1000,800,600,400,200,100], carrying_cap=10**6, timesteps=3000))
box_switching(pairs[2], frequencies=[1000,800,600,400,200,100], carrying_cap=10**6, timesteps=3000)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
1000,96,0.692308,0.260417,"0000,0010,0011,0111"
800,91,0.428571,0.274725,"0000,0100,0110,0111"
600,96,0.750000,0.250000,"0000,0010,0011,0111"
400,93,0.647059,0.268817,"0000,0100,0110,0111"
200,100,1.000000,0.360000,"0000,0100,0110,0111"
100,98,0.800000,0.418367,"0000,0100,0110,0111"


In [27]:
plot_simulation(simulate(dataset2.loc['SAM'].tolist(), carrying_cap=10**6, timesteps=3000))

In [13]:
pathway_analysis(dataset2.loc['SAM'], pretty=True, carrying_cap=10**6)

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0001,1001,1101,1111",Yes,Yes,4,0.114286
1,"0000,0001,1001,1011,1111",Yes,No,6,0.171429
2,"0000,0100,1100,1101,1111",Yes,No,5,0.142857
3,"0000,0001",No,No,6,nan
4,"0000,0100",No,No,7,nan
5,"0000,1000,1100,1101,1111",Yes,No,9,0.257143
6,"0000,0001,1001",No,No,12,nan
7,"0000,0001,1001,1101",No,No,8,nan
8,"0000,1000,1001,1011,1111",Yes,No,4,0.0857143
9,"0000,1000,1010,1110",No,No,1,nan


Global optimum: 1111

Weight of the dominant successful pathway: 0.2571428571428571

Success rate: 0.4090909090909091

### K=10^4

In [18]:
display(table_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,100,1.0,0.81,"0000,0010,0011,0111"
175,100,1.0,0.80,"0000,0010,0011,0111"
150,100,1.0,0.80,"0000,0010,0011,0111"
125,100,1.0,0.79,"0000,0010,0011,0111"
100,100,1.0,0.84,"0000,0010,0011,0111"
75,100,1.0,0.80,"0000,0010,0011,0111"
50,100,1.0,0.82,"0000,0010,0011,0111"
25,100,1.0,0.85,"0000,0010,0011,0111"
20,100,1.0,0.86,"0000,0010,0011,0111"


## Carrying capacities below 10^4 do not succeed

The tables below are for K=10^3:

In [19]:
for pair in pairs:
    display(Markdown('__{} + {}__'.format(pair[0].name, pair[1].name)))
    display(table_switching(pair, carrying_cap=10**3, prob_mutation=10**-2))

__CTX + SAM__

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,0,0.000000,NaN,
175,0,0.000000,NaN,
150,0,0.000000,NaN,
125,0,0.000000,NaN,
100,0,0.000000,NaN,
75,0,0.000000,NaN,
50,0,0.000000,NaN,
25,0,0.000000,NaN,
20,0,0.000000,NaN,


__AM + AMC__

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,0,0.0,NaN,
175,0,0.0,NaN,
150,0,0.0,NaN,
125,0,0.0,NaN,
100,0,0.0,NaN,
75,0,0.0,NaN,
50,0,0.0,NaN,
25,0,0.0,NaN,
20,0,0.0,NaN,


__ZOX + CXM__

,Success count,Success rate,Weight of dominant path,Dominant path
f,,,,
200,0,0.0,NaN,
175,0,0.0,NaN,
150,0,0.0,NaN,
125,0,0.0,NaN,
100,0,0.0,NaN,
75,0,0.0,NaN,
50,0,0.0,NaN,
25,0,0.0,NaN,
20,0,0.0,NaN,


## Is switching at a frequency of 1 the same as averaging 2 landscapes?

In [20]:
def avg_landscape(pair):
    return [(i + j) / 2 for i, j in zip(pair[0], pair[1])]
display(Markdown('### Average of ZOX and CXM'))
plot_simulation(simulate(avg_landscape(pairs[2])))
pathway_analysis(pd.Series(avg_landscape(pairs[2]), name='avg'), pretty=True)
display(Markdown('### Switching between ZOX and CXM at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[2]], frequency=1))
pathway_analysis(pairs[2], pretty=True, frequency=1)

### Average of ZOX and CXM

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0100,0110,0111",Yes,Yes,98,0.98
1,"0000,0010,0110,0111",Yes,No,2,0.02


Global optimum: 0111

Weight of the dominant successful pathway: 0.98

Success rate: 1.0

### Switching between ZOX and CXM at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010,0011,0111",No,Yes,0,nan
1,"0000,0100,0110,0111",Yes,No,98,0.98
2,"0000,0010,0110,0111",Yes,No,2,0.02


Global optimum: 0111

Weight of the dominant successful pathway: 0.98

Success rate: 1.0

They are pretty similar!

In [21]:
display(Markdown('### Average of AM and AMC'))
plot_simulation(simulate(avg_landscape(pairs[1]), carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pd.Series(avg_landscape(pairs[1]), name='avg'), pretty=True, carrying_cap=10**6, prob_mutation=10**-5)
display(Markdown('### Switching between AM and AMC at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[1]], frequency=1, carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pairs[1], pretty=True, frequency=1, carrying_cap=10**6, prob_mutation=10**-5)

### Average of AM and AMC

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0001,1001,1101",Yes,Yes,90,0.9
1,"0000,0001,0011,1011,1111,1101",Yes,No,2,0.02
2,"0000,1000,1001,1101",Yes,No,2,0.02
3,"0000,0001,1001,1011,1111,1101",Yes,No,3,0.03
4,"0000,0001,0101,1101",Yes,No,3,0.03


Global optimum: 1101

Weight of the dominant successful pathway: 0.9

Success rate: 1.0

### Switching between AM and AMC at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010",No,Yes,0,nan
1,"0000,0001,1001,1101",Yes,No,87,0.87
2,"0000,0001,0101,1101",Yes,No,5,0.05
3,"0000,1000,1001,1101",Yes,No,3,0.03
4,"0000,0001,1001,1011,1111,1101",Yes,No,4,0.04
5,"0000,0001,0011,1011,1111,1101",Yes,No,1,0.01


Global optimum: 1101

Weight of the dominant successful pathway: 0.87

Success rate: 1.0